In [ ]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# --- Nastavení ---
train_dirs = ["data2"]   # složky s třídami
test_dirs  = ["data1", "data2"]  # složky pro test
image_size = (224, 224)
batch_size = 16
AUTOTUNE = tf.data.AUTOTUNE
ALLOWED_EXT = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".gif")

In [ ]:
# --- 1) Najdu třídy a štítky ---
train_classes = sorted({
    cls for d in train_dirs
    for cls in os.listdir(d)
    if os.path.isdir(os.path.join(d, cls))
})
label_dict = {cls: idx for idx, cls in enumerate(train_classes)}

all_sets = [set(train_classes)]
for d in test_dirs:
    all_sets.append({
        cls for cls in os.listdir(d)
        if os.path.isdir(os.path.join(d, cls))
    })
common_classes = sorted(set.intersection(*all_sets))

def get_filepaths_and_labels(dirs, classes, label_dict):
    paths, labels = [], []
    for cls in classes:
        for base in dirs:
            class_path = os.path.join(base, cls)
            if not os.path.isdir(class_path):
                continue
            for fname in os.listdir(class_path):
                if not fname.lower().endswith(ALLOWED_EXT):
                    continue
                fpath = os.path.join(class_path, fname)
                if os.path.isfile(fpath):
                    paths.append(fpath)
                    labels.append(label_dict[cls])
    return paths, labels

train_paths, train_labels = get_filepaths_and_labels(train_dirs, train_classes, label_dict)
test_paths,  test_labels  = get_filepaths_and_labels(test_dirs,  common_classes,   label_dict)

In [ ]:
# --- 2) Důkladná kontrola integrity obrázků ---
def filter_valid_images(paths, labels):
    valid_paths, valid_labels = [], []
    for p, lbl in zip(paths, labels):
        try:
            # 1) ověření hlavičky
            with Image.open(p) as im:
                im.verify()
            # 2) skutečné načtení dat
            with Image.open(p) as im:
                im.load()
            valid_paths.append(p)
            valid_labels.append(lbl)
        except Exception:
            print(f"Vynechávám poškozený nebo nekompatibilní soubor: {p}")
    return valid_paths, valid_labels

train_paths, train_labels = filter_valid_images(train_paths, train_labels)
test_paths,  test_labels  = filter_valid_images(test_paths,  test_labels)

In [ ]:
# --- 3) Sestavení tf.data pipeline ---
def parse_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, image_size)
    img = preprocess_input(img)
    return img, tf.one_hot(label, len(train_classes))

train_ds = (
    tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    .shuffle(len(train_paths))
    .map(parse_and_preprocess, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
    .map(parse_and_preprocess, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
# --- 4) Model s ResNet50 ---
inp = Input(shape=(*image_size, 3))
base = ResNet50(weights="imagenet", include_top=False, input_tensor=inp)
base.trainable = False  # jen hlavička se trénuje

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.5)(x)
out = Dense(len(train_classes), activation="softmax")(x)

model = Model(inp, out)
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# --- 5) Trénink ---
model.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds,
    verbose=1
)

In [ ]:
# --- 6) Vyhodnocení ---
train_score = model.evaluate(train_ds, verbose=0)
test_score  = model.evaluate(test_ds,  verbose=0)
print(f"Train accuracy: {train_score[1]:.4f}")
print(f" Test accuracy: {test_score[1]:.4f}")

In [ ]:
# --- 7) Konfuzní matice ---

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Předpovědi modelu (pravděpodobnosti)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Skutečné hodnoty ze vstupu
y_true = np.concatenate([np.argmax(y.numpy(), axis=1) for _, y in test_ds])

# Výpočet konfuzní matice
cm = confusion_matrix(y_true, y_pred)

# Zobrazení
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=common_classes, yticklabels=common_classes)
plt.xlabel("Predikovaná třída")
plt.ylabel("Skutečná třída")
plt.title("Konfuzní matice")
plt.show()